In [1]:
import wandb
run = wandb.init()
artifact = run.use_artifact('ain-space/gaia/model-85h3dcq1:v0', type='model')
artifact_dir = artifact.download()

wandb: Currently logged in as: bernddoser. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb:   1 of 1 files downloaded.  


In [2]:
import torch
import onnxscript
from spherinator.models import (
    DenseModel,
    VariationalAutoencoderPure,
)

encoder = DenseModel(layer_dims=[110, 55, 4])
decoder = DenseModel(layer_dims=[3, 55, 110])
model = VariationalAutoencoderPure(encoder=encoder, decoder=decoder)

ckpt = "artifacts/model-85h3dcq1:v0/model.ckpt"
checkpoint = torch.load(ckpt, weights_only=True, map_location="cpu")
model.load_state_dict(checkpoint["state_dict"])
model.eval()

export_options = torch.onnx.ExportOptions(dynamic_shapes=True)
onnx = torch.onnx.dynamo_export(model.variational_encoder, torch.randn(10, 110, device="cpu"), export_options=export_options)
onnx.save("vae_encoder.onnx")

onnx = torch.onnx.dynamo_export(model.decoder, torch.randn(10, 3, device="cpu"), export_options=export_options)
onnx.save("vae_decoder.onnx")

/home/doserbd/.cache/pypoetry/virtualenvs/spherinator-zzYPp9oG-py3.12/lib/python3.12/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/doserbd/.cache/pypoetry/virtualenvs/spherinator-zzYPp9oG-py3.12/lib/python3.12/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/doserbd/.cache/pypoetry/virtualenvs/spherinator-zzYPp9oG-py3.12/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
/home/doserbd/.cache/pyp